# 기말고사 예상문제 (ver 0.1)
> 

- toc:true
- branch: master
- badges: true
- comments: false
- author: 최서연

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import requests

## `#1`. COVID19 시도별 접종률 시각화

아래의 그림은 COVID19 예방접종의 시도별 현황을 캡쳐한 것이다. 

![](https://github.com/guebin/2021DV/blob/master/_notebooks/final_FIG1.png?raw=true "그림1: COVID19 예방접종의 시도별 현황")

이 정보는 특정 주기로 업데이트 되며 아래의 웹페이지 2번째 테이블에서 확인할 수 있다. 
```
https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000
```
판다스의 `pd.read_html()` 함수를 이용해 위의 페이지에서 그림1의 테이블을 읽어오라. 그리고 folium의 choroplethmap을 활용하여 시도별 2차접종의 접종률을 시각화 하라. 시각화 예시는 그림2와 같다. 

---

**힌트1.** pd.read_html()을 이용할때 `encoding='utf-8'`옵션을 사용하라. 

**힌트2.** `https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000`의 2번째 테이블을 판다스로 읽어온 결과는 아래와 같아야 한다. 

In [2]:
df=pd.read_html('https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000',encoding='utf-8')[1]
df

구분   1차접종             2차접종              3차접종         
    구분  당일 실적     당일 누계  당일 실적     당일 누계   당일 실적    당일 누계
0   합계  54184  43104019  43060  41807882  787801  7937480
1   서울   8483   8020048   6705   7805297  139233  1489010
2   부산   4024   2757462   3224   2670259   58699   509902
3   대구   2824   1918142   2147   1856244   30643   297445
4   인천   2772   2458922   2404   2384813   40491   426391
5   광주   1713   1204447   1685   1164710   23536   215904
6   대전   1844   1187726   1555   1149300   20177   202449
7   울산   1410    919495    866    890888   15672   135302
8   세종    374    276403    321    266137    3831    44602
9   경기  12932  11285264   9217  10946077  173172  1925100
10  강원   1717   1292805   1638   1256039   28893   271703
11  충북   1593   1365516   1479   1325872   26614   265445
12  충남   2250   1809005   2064   1752184   36225   365771
13  전북   2072   1531118   1705   1488334   39020   356320
14  전남   1870   1590099   1799   1544293   39305   390707
15  경북   3007   2182756   2401   2113629   47067   429860
16  경남   4388   2747064   3234   2656146   55766   518898
17  제주    911    557747    616    537660    9457    92671

In [3]:
# df.iloc[1:,:].columns

In [4]:
# df.iloc[1:,:].set_index((       '구분',    '구분')).stack().reset_index().query('level_1=="당일 누계"').set_index((       '구분',    '구분')).iloc[:,2]

In [5]:
df.iloc[1:].set_index((       '구분',    '구분')).stack().reset_index().iloc[:,[0,1,3]].query('level_1=="당일 누계"').iloc[:,[0,2]].reset_index().iloc[:,[1,2]]

,"(구분, 구분)",2차접종
0,서울,7805297
1,부산,2670259
2,대구,1856244
3,인천,2384813
4,광주,1164710
5,대전,1149300
6,울산,890888
7,세종,266137
8,경기,10946077
9,강원,1256039


In [6]:
_df=_

In [7]:
_df.columns=['city','vaccination']

**힌트3.** 아래의 주소에서 json을 파일을 읽어온뒤 choroplethmap을 그릴때 이용하라. 
```
https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json
```

In [8]:
global_dict = json.loads(requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json').text)

**힌트4.** json파일의 지역명과 `https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000`의 지역이름을 맞추어라. json파일의 지역명은 아래와 같은 코드로 확인할 수 있다. 
```python
[global_dict['features'][i]['properties']['name'] for i in range(17)]
```
```
['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']
```

- 여기에서 `global_dict`는 json파일을 dict의 형태로 저장한 변수명이다. 

**힌트5.** 지역별 접종률은 아래의 수식으로 구한다. 

$$
\textbf{서울특별시 2차접종 접종률}=\frac{\textbf{서울특별시 2차접종 당일누계}}{\textbf{서울특별시 인구수}}
$$

이때 위의 수식의 분모에 사용할 시도별 인구수는 아래의 주소에서 얻을 수 있다. 
```
https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/2021-11-22-prov.csv
```

In [9]:
df2=pd.read_csv('https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/2021-11-22-prov.csv')
df2.columns=['city','population']
# df2

In [10]:
# [_df['vaccination'].iloc[i]/df2['population'][i] for i in range(17)]

In [11]:
_df2=pd.DataFrame({'key':(df2.iloc[:,0]),'value':[_df['vaccination'].iloc[i]/df2['population'][i] for i in range(17)]})
#_df2

In [12]:
_df3=pd.DataFrame({'key':(df2.iloc[:,0]),'vaccination':_df['vaccination'],'population':df2['population']})

In [13]:
#_df3.assign(z=list(map(lambda x,y: x/y, _df3.vaccination,_df3.population)))

In [14]:
#_df3.eval('value=vaccination/population')

위 세 경우 모두 같은 결과!!!

**힌트6.** 시도별 접종률의 결과는 아래와 같다. 

In [15]:
m = folium.Map([35.82894918383515, 127.15323521515239],zoomstart=2,scrollWheelZoom=False)
choro = folium.Choropleth(
    data=_df3.eval('value=vaccination/population'), 
    geo_data=global_dict,
    columns=['key','value'],
    key_on = 'feature.properties.name'
)
choro.add_to(m)
#m

## `#2`. COVID19 시도별/월별 확산과정 시각화

아래는 COVID19 확진자수를 지역별로 매일 기록한 자료이다. 
```
https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/covid19_20211202.csv
```
자료를 판다스로 불러온 결과는 아래와 같다. 

일별로 기록된 COVID19 확진자수를 월별로 통합한 뒤 `2021-01 ~ 2021-10` 기간의 발생률을 계산하여 시각화하라. 시각화는 `plotly`의 `choropleth_mapbox`를 이용하며 시간의 추이를 표현하기 위해 `animation_frame` 옵션을 사용한다. 시각화 예시는 아래와 같다. 

In [16]:
df=pd.read_csv('https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/covid19_20211202.csv')

In [17]:
df.query('일자>"2021-01-01" and 일자<"2021-10-31"').reset_index().iloc[:,1:]

,일자,계(명),서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주,검역
0,2021-01-02,820,250,26,29,71,13,4,17,3,227,31,36,7,11,11,20,37,13,14
1,2021-01-03,657,198,19,22,48,26,8,7,1,204,33,27,9,5,12,14,12,4,8
2,2021-01-04,"1,020",329,37,29,103,74,6,5,-,268,28,23,23,13,3,25,33,10,11
3,2021-01-05,714,199,15,32,49,23,8,4,-,230,27,32,18,5,4,24,21,8,15
4,2021-01-06,838,263,38,21,38,30,9,2,1,284,16,21,24,22,1,28,29,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2021-10-26,"1,265",427,33,61,74,5,13,9,6,431,25,31,40,41,6,20,29,6,8
298,2021-10-27,"1,952",729,43,39,125,9,15,8,1,740,19,29,59,30,6,42,45,7,6
299,2021-10-28,"2,110",788,52,106,137,7,6,6,2,707,22,54,82,41,12,41,35,4,8
300,2021-10-29,"2,124",770,43,117,132,5,9,4,1,762,21,35,68,56,13,42,34,4,8


```python
'2021-01-31'.split('-')[0]+'-'+'2021-01-31'.split('-')[1]
```
```
'2021-01'
```


In [18]:
_df=_

In [19]:
# def f(x): 
#     if x<"2021-01-31": y='2021-01' 
#     elif x<"2021-02-28": y='2021-02'
#     elif x<"2021-03-31": y='2021-03'
#     elif x<"2021-04-30": y='2021-04'
#     elif x<"2021-05-31": y='2021-05'
#     elif x<"2021-06-30": y='2021-06'
#     elif x<"2021-07-31": y='2021-07'
#     elif x<"2021-08-31": y='2021-08'
#     elif x<"2021-09-30": y='2021-09'
#     else:  y='2021-10'
#     return y 

In [20]:
_df['ym']=list(map(lambda x: x.split('-')[0]+'-'+x.split('-')[1],df.query('일자>"2021-01-01" and 일자<"2021-10-31"').reset_index().iloc[:,1:].일자))
_df

,일자,계(명),서울,부산,대구,인천,광주,대전,울산,세종,...,강원,충북,충남,전북,전남,경북,경남,제주,검역,ym
0,2021-01-02,820,250,26,29,71,13,4,17,3,...,31,36,7,11,11,20,37,13,14,2021-01
1,2021-01-03,657,198,19,22,48,26,8,7,1,...,33,27,9,5,12,14,12,4,8,2021-01
2,2021-01-04,"1,020",329,37,29,103,74,6,5,-,...,28,23,23,13,3,25,33,10,11,2021-01
3,2021-01-05,714,199,15,32,49,23,8,4,-,...,27,32,18,5,4,24,21,8,15,2021-01
4,2021-01-06,838,263,38,21,38,30,9,2,1,...,16,21,24,22,1,28,29,5,6,2021-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2021-10-26,"1,265",427,33,61,74,5,13,9,6,...,25,31,40,41,6,20,29,6,8,2021-10
298,2021-10-27,"1,952",729,43,39,125,9,15,8,1,...,19,29,59,30,6,42,45,7,6,2021-10
299,2021-10-28,"2,110",788,52,106,137,7,6,6,2,...,22,54,82,41,12,41,35,4,8,2021-10
300,2021-10-29,"2,124",770,43,117,132,5,9,4,1,...,21,35,68,56,13,42,34,4,8,2021-10


In [21]:
_df.set_index('ym').iloc[:,2:19].applymap(lambda x: x.replace(',','') if x!='-' else None).applymap(lambda x: float(x) if x!=None else None).groupby('ym').agg(np.mean).T

ym,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10
서울,160.100000,145.714286,122.387097,193.566667,196.064516,202.400000,463.354839,483.903226,704.466667,611.566667
부산,27.733333,16.642857,18.806452,41.466667,19.967742,18.433333,64.032258,102.580645,44.500000,37.733333
대구,15.466667,11.250000,9.806452,14.133333,18.225806,22.733333,34.548387,63.774194,61.766667,52.866667
인천,30.300000,22.071429,20.225806,20.233333,17.193548,18.966667,73.967742,89.258065,132.500000,113.800000
광주,22.833333,10.750000,4.032258,7.407407,12.774194,4.400000,14.903226,21.096774,31.200000,11.966667
대전,8.925926,4.043478,4.000000,14.800000,11.225806,19.100000,49.612903,47.032258,44.600000,17.866667
울산,7.620690,3.166667,5.222222,24.166667,22.548387,8.785714,12.322581,31.258065,33.000000,8.366667
세종,2.421053,1.764706,3.142857,3.346154,4.920000,3.076923,5.866667,8.290323,8.200000,4.066667
경기,168.966667,133.821429,162.225806,187.233333,157.580645,165.933333,378.096774,493.483871,634.866667,587.733333
강원,15.266667,6.035714,15.032258,11.800000,16.161290,12.466667,30.129032,31.193548,32.566667,24.633333


In [22]:
_df2=_

---

In [23]:
global_dict=json.loads(requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json').text)

**힌트1.** 아래의 주소에서 json을 파일을 읽어온뒤 choroplethmap을 그릴때 이용하라. 
```
https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json
```

**힌트2.** json파일의 지역명과 `https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000`의 지역이름을 맞추어라. json파일의 지역명은 아래와 같은 코드로 확인할 수 있다. 
```python
[global_dict['features'][i]['properties']['name'] for i in range(17)]
```
```
['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']
```

- 여기에서 `global_dict`는 힌트1의 json파일이 저장된 dictionary이다. 

**힌트3.** 각 지역의 월별 발생률은 아래의 수식으로 구한다. 

$$
\textbf{2021-01 서울특별시 코로나 발생률}=\frac{\textbf{2021-01 서울특별시 코로나 확진자수 총합}}{\textbf{서울특별시 인구수}}
$$

이때 위의 수식의 분모에 사용할 시도별 인구수는 아래의 주소에서 얻을 수 있다. 
```
https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/2021-11-22-prov.csv
```

In [24]:
pop=pd.read_csv('https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/2021-11-22-prov.csv')
pop.columns=['prov','pop']

In [25]:
pop

,prov,pop
0,서울특별시,9532428
1,부산광역시,3356311
2,대구광역시,2390721
3,인천광역시,2945009
4,광주광역시,1442454
5,대전광역시,1454228
6,울산광역시,1122566
7,세종특별자치시,368276
8,경기도,13549577
9,강원도,1537717


In [26]:
pop.prov

0       서울특별시
1       부산광역시
2       대구광역시
3       인천광역시
4       광주광역시
5       대전광역시
6       울산광역시
7     세종특별자치시
8         경기도
9         강원도
10       충청북도
11       충청남도
12       전라북도
13       전라남도
14       경상북도
15       경상남도
16    제주특별자치도
Name: prov, dtype: object

In [27]:
_df2.reset_index().assign(index=pop.prov).rename(columns={'index':'prov'}).merge(pop)

,prov,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,pop
0,서울특별시,160.100000,145.714286,122.387097,193.566667,196.064516,202.400000,463.354839,483.903226,704.466667,611.566667,9532428
1,부산광역시,27.733333,16.642857,18.806452,41.466667,19.967742,18.433333,64.032258,102.580645,44.500000,37.733333,3356311
2,대구광역시,15.466667,11.250000,9.806452,14.133333,18.225806,22.733333,34.548387,63.774194,61.766667,52.866667,2390721
3,인천광역시,30.300000,22.071429,20.225806,20.233333,17.193548,18.966667,73.967742,89.258065,132.500000,113.800000,2945009
4,광주광역시,22.833333,10.750000,4.032258,7.407407,12.774194,4.400000,14.903226,21.096774,31.200000,11.966667,1442454
5,대전광역시,8.925926,4.043478,4.000000,14.800000,11.225806,19.100000,49.612903,47.032258,44.600000,17.866667,1454228
6,울산광역시,7.620690,3.166667,5.222222,24.166667,22.548387,8.785714,12.322581,31.258065,33.000000,8.366667,1122566
7,세종특별자치시,2.421053,1.764706,3.142857,3.346154,4.920000,3.076923,5.866667,8.290323,8.200000,4.066667,368276
8,경기도,168.966667,133.821429,162.225806,187.233333,157.580645,165.933333,378.096774,493.483871,634.866667,587.733333,13549577
9,강원도,15.266667,6.035714,15.032258,11.800000,16.161290,12.466667,30.129032,31.193548,32.566667,24.633333,1537717


In [28]:
_df2.reset_index().assign(index=pop.prov).rename(columns={'index':'prov'}).merge(pop).melt(id_vars=('prov','pop')).eval('prop=value/pop')

,prov,pop,variable,value,prop
0,서울특별시,9532428,2021-01,160.100000,0.000017
1,부산광역시,3356311,2021-01,27.733333,0.000008
2,대구광역시,2390721,2021-01,15.466667,0.000006
3,인천광역시,2945009,2021-01,30.300000,0.000010
4,광주광역시,1442454,2021-01,22.833333,0.000016
...,...,...,...,...,...
165,전라북도,1789770,2021-10,21.700000,0.000012
166,전라남도,1834653,2021-10,14.300000,0.000008
167,경상북도,2627925,2021-10,44.333333,0.000017
168,경상남도,3318161,2021-10,49.566667,0.000015


In [29]:
#_df2=_

In [30]:
#pop.merge(_df2).melt(id_vars=('prov','pop')).eval('prop=value/pop')

In [31]:
_df3=_

In [32]:
_df3

,prov,pop,variable,value,prop
0,서울특별시,9532428,2021-01,160.100000,0.000017
1,부산광역시,3356311,2021-01,27.733333,0.000008
2,대구광역시,2390721,2021-01,15.466667,0.000006
3,인천광역시,2945009,2021-01,30.300000,0.000010
4,광주광역시,1442454,2021-01,22.833333,0.000016
...,...,...,...,...,...
165,전라북도,1789770,2021-10,21.700000,0.000012
166,전라남도,1834653,2021-10,14.300000,0.000008
167,경상북도,2627925,2021-10,44.333333,0.000017
168,경상남도,3318161,2021-10,49.566667,0.000015


In [33]:
_df3.columns=['prov','pop','ym','confirmed','prop']

**힌트4**. `3,925`와 같은 문자열을 처리하기 위해 아래를 참고하라. 

In [34]:
'3,925'.replace(',','')

'3925'

**힌트5.** 시각화에 사용할 데이터프레임의 형태 및 핵심정보는 아래와 같다. 

In [35]:
#hide_input
from IPython.display import HTML
HTML('<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>ym</th>\n      <th>prov</th>\n      <th>confirmed</th>\n      <th>pop</th>\n      <th>prop</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>2021-01</td>\n      <td>강원도</td>\n      <td>488.0</td>\n      <td>1537717</td>\n      <td>0.000317</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>2021-02</td>\n      <td>강원도</td>\n      <td>169.0</td>\n      <td>1537717</td>\n      <td>0.000110</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>2021-03</td>\n      <td>강원도</td>\n      <td>466.0</td>\n      <td>1537717</td>\n      <td>0.000303</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>2021-04</td>\n      <td>강원도</td>\n      <td>354.0</td>\n      <td>1537717</td>\n      <td>0.000230</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>2021-05</td>\n      <td>강원도</td>\n      <td>501.0</td>\n      <td>1537717</td>\n      <td>0.000326</td>\n    </tr>\n    <tr>\n      <th>...</th>\n      <td>...</td>\n      <td>...</td>\n      <td>...</td>\n      <td>...</td>\n      <td>...</td>\n    </tr>\n    <tr>\n      <th>165</th>\n      <td>2021-06</td>\n      <td>충청북도</td>\n      <td>363.0</td>\n      <td>1596948</td>\n      <td>0.000227</td>\n    </tr>\n    <tr>\n      <th>166</th>\n      <td>2021-07</td>\n      <td>충청북도</td>\n      <td>544.0</td>\n      <td>1596948</td>\n      <td>0.000341</td>\n    </tr>\n    <tr>\n      <th>167</th>\n      <td>2021-08</td>\n      <td>충청북도</td>\n      <td>1302.0</td>\n      <td>1596948</td>\n      <td>0.000815</td>\n    </tr>\n    <tr>\n      <th>168</th>\n      <td>2021-09</td>\n      <td>충청북도</td>\n      <td>1192.0</td>\n      <td>1596948</td>\n      <td>0.000746</td>\n    </tr>\n    <tr>\n      <th>169</th>\n      <td>2021-10</td>\n      <td>충청북도</td>\n      <td>1556.0</td>\n      <td>1596948</td>\n      <td>0.000974</td>\n    </tr>\n  </tbody>\n</table>')

,ym,prov,confirmed,pop,prop
0,2021-01,강원도,488.0,1537717,0.000317
1,2021-02,강원도,169.0,1537717,0.000110
2,2021-03,강원도,466.0,1537717,0.000303
3,2021-04,강원도,354.0,1537717,0.000230
4,2021-05,강원도,501.0,1537717,0.000326
...,...,...,...,...,...
165,2021-06,충청북도,363.0,1596948,0.000227
166,2021-07,충청북도,544.0,1596948,0.000341
167,2021-08,충청북도,1302.0,1596948,0.000815
168,2021-09,충청북도,1192.0,1596948,0.000746


**힌트 6**. 데이터프레임이 힌트5와 유사한 형태로 정리되었다는 가정하에 적절한 시각화 코드는 아래와 같다. (여기에서 `df`는 힌트5의 데이터프레임, `global_dict`는 힌트1~2의 json파일이 저장된 dictionary 이다) 

```python
import plotly.express as px
from IPython.display import HTML
fig=px.choropleth_mapbox(df, 
                     geojson=global_dict, 
                     color=??,
                     locations=?? 
                     animation_frame='ym',
                     featureidkey=??,
                     center={"lat": 36, "lon": 128},
                     mapbox_style="carto-positron", 
                     range_color=(0, df.prop.max()),
                     height=1200,
                     zoom=6.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
HTML(fig.to_html(include_mathjax=False, config=dict({'scrollZoom':False})))
```

- 주의: 컴퓨터 사양에 따라 시각화이후 컴퓨터가 심하게 느려질 수 있는데 이 경우 반드시 주피터 노트북에 시각화 결과를 임베딩하여 제출할 필요가 없다. 즉 아래의 같이 시각화를 수행하는 코드를 따로 주석처리 하여 제출하여도 무방하다. (의미가 이해안되면 질문하세요.) 

```python
import plotly.express as px
from IPython.display import HTML
fig=px.choropleth_mapbox(df, 
                     geojson=global_dict, 
                     color=??,
                     locations=?? 
                     animation_frame='ym',
                     featureidkey=??,
                     center={"lat": 36, "lon": 128},
                     mapbox_style="carto-positron", 
                     range_color=(0, df.prop.max()),
                     height=1200,
                     zoom=6.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
_html=fig.to_html(include_mathjax=False, config=dict({'scrollZoom':False}))
#HTML(_html)
```

In [36]:
_df3.columns

Index(['prov', 'pop', 'ym', 'confirmed', 'prop'], dtype='object')

my answer
`````python
import plotly.express as px
from IPython.display import HTML
fig=px.choropleth_mapbox(_df3, 
                     geojson=global_dict, 
                     color='prop',
                     locations='prov' ,
                     animation_frame='ym',
                     featureidkey='properties.name',
                     center={"lat": 36, "lon": 128},
                     mapbox_style="carto-positron", 
                     range_color=(0, _df3.prop.max()),
                     height=1200,
                     zoom=6.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
HTML(fig.to_html(include_mathjax=False, config=dict({'scrollZoom':False})))
`````

**힌트 7**. 아래의 자료에서 `-`는 확진자가 없다는 의미이므로 0으로 생각해도 무방하다. 